In [32]:
from khmernltk import word_tokenize
import unicodedata
import re
import pandas as pd



In [33]:
data = pd.read_csv("../data/kh-polar.ver1.0.csv")
data.head()

,text,keyword,label
0,នាយិកាមជ្ឈមណ្ឌលសិទ្ធិមនុស្សកម្ពុជាអ្នកស្រី ចក់...,បង្ហាញខ្លួន,neutral
1,ការឃុំខ្លួនកញ្ញា សេង ធារី កាន់តែយូរដោយរដ្ឋាភិប...,ដឹងឮកាន់តែខ្លាំង,positive
2,ប្រភពបង្ហើបថា បន្ទប់នោះមានខ្ទង់ចំណាយប្រមាណជាង ...,បង្ហើប,neutral
3,1956បានបង្ហាញថាផូស្វ័របានផ្ទេរចេញពីដើមបែកអារទៅ...,ផ្ទេរ,neutral
4,ដរាបណាយើងមិនបានតាំងចិត្តខិតខំប្រឹងរៀន ប្រឹងធ្វ...,មិនបានតាំងចិត្តខិតខំប្រឹងរៀន/គ្មានថ្ងៃក្លាយជារ...,negative


In [34]:
data = data.drop(data.columns[1], axis=1)
data.head()

,text,label
0,នាយិកាមជ្ឈមណ្ឌលសិទ្ធិមនុស្សកម្ពុជាអ្នកស្រី ចក់...,neutral
1,ការឃុំខ្លួនកញ្ញា សេង ធារី កាន់តែយូរដោយរដ្ឋាភិប...,positive
2,ប្រភពបង្ហើបថា បន្ទប់នោះមានខ្ទង់ចំណាយប្រមាណជាង ...,neutral
3,1956បានបង្ហាញថាផូស្វ័របានផ្ទេរចេញពីដើមបែកអារទៅ...,neutral
4,ដរាបណាយើងមិនបានតាំងចិត្តខិតខំប្រឹងរៀន ប្រឹងធ្វ...,negative


In [35]:

# Load stopwords
stopwords_df = pd.read_csv("../stop_word/khmer_stopwords.csv", header=None)  # if it has no header
stopwords = set(stopwords_df[0].tolist())  # convert to a set for faster lookup
print(stopwords)

{'ឬ', 'សម្រាប់', 'ខ្លួនឯង', 'ទទេ', 'ពី', 'គាត់', 'ដំបូង', 'ទៅដល់', 'ម្ដង', 'វា', 'ទីពីរ', 'គ្នា', 'នោះ', 'ក្នុង', 'ទទួល', 'ពីនេះតទៅ', 'ជំរាល', 'ភាព', 'នៅទី', 'គ្មានមួយ', 'កំពូល', 'ដូចនេះហើយ', 'ផ្ទុយនឹង', 'ន័យនេះ', 'អ្នក', 'ម្ភៃ', 'រាល់', 'ឥត', 'ប៉ុនគ្នា', 'ពេលនោះ', 'ដោយទីពីរ', 'សព្វ', 'ត្រឹមតែ', 'តាម', 'យ៉ាងនេះ', 'បន្ទាប់ពី', 'ចំនួនច្រើន', 'ទាស់', 'គឺជា', 'ដោយមិនដឹងជារឿងអ្វី', 'ដូចជា', 'នេះ', 'សោះ', 'ថ្មី', 'ឯកោ', 'ចេញ', 'ដង', 'ញែក', 'មួយទៀត', 'នេះទៅទៀត', 'ខុស', 'ទាំង', 'នរណា', 'បន្ទាប់ពីនេះ', 'ស្ងៀម', 'យ៉ាង', 'ម្នាក់', 'កាន់', 'សំរាប់', 'និមួយ', 'ញឹកញាប់', 'ដោយហេតុនោះ', 'តែមួយ', 'ទៀត', 'ឡើយ', 'ដល់ម្ល៉េះ', 'ខ្ងុំ', 'យក', 'ទៅផុត', 'ទាំងនេះ', 'ព្រោះ', 'លោក', 'ចង់', 'ហុកសិប', 'ដែលក្លាយ', 'ដោយមិនដឹងជាយ៉ាងម៉េច', 'ក្នុងនេះ', 'ពីព្រោះ', 'ទូទាំង', 'យល់ស្រប', 'ទទឹង', 'ក្រោម', 'សេចក្ដី', 'ថា', 'ម្ល៉េះ', 'ចេញពី', 'ប្រហែល', 'ក្រាស់', 'រក្សា', 'ចុងក្រោយ', 'បែបនេះ', 'តាំងពីនោះ', 'កាលនោះ', 'របស់លោក', 'លើក', 'ដែលក្រោយបំផុត', 'គឺ', 'ប្រាំ', 'កាលណា', 'ខាងលើ', 'គួរសម', 'ពួកគេ', 'ពីនេះពីនោះ', 'រយះពេល', 'ក

In [36]:
# Define your emoji & punctuation pattern
pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoji (emoticons)
    "\U0001F300-\U0001F5FF"  # emoji (symbols & pictographs)
    "\U0001F680-\U0001F6FF"  # emoji (transport)
    "\U0001F1E0-\U0001F1FF"  # emoji (flags)
    ".,!?;:()\"'។"          # punctuation + Khmer full stop
    "]",
    flags=re.UNICODE
)


def clean_text(text):
    # 1. Unicode normalization
    text = unicodedata.normalize("NFC", str(text))
    
    # 1b. Convert to lowercase to handle English letters
    text = text.lower()
    
    # 2. Remove emojis and punctuation
    text = pattern.sub("", text)
    
    # 3. Tokenize Khmer text
    tokens = word_tokenize(text)
    
    # 4. Remove stopwords AND empty tokens
    filtered_tokens = [word for word in tokens if word not in stopwords and word.strip() != ""]
    
    # Join back to string
    return " ".join(filtered_tokens)



In [37]:
# Apply to all rows
data['text'] = data['text'].apply(clean_text)

In [38]:
data.head()

,text,label
0,នាយិកា មជ្ឈមណ្ឌល សិទ្ធិ មនុស្ស កម្ពុជា អ្នកស្រ...,neutral
1,ការឃុំ កញ្ញា សេង ធារី កាន់តែ យូរ រដ្ឋាភិបាល ហ៊...,positive
2,ប្រភព បង្ហើប បន្ទប់ ខ្ទង់ ចំណាយ ជាង ១០ម៉ឺន ដុល...,neutral
3,1956 បាន បង្ហាញ ផូស្វ័រ បាន ផ្ទេរ ដើម បែក អារ ...,neutral
4,ដរាបណា មិន បាន តាំងចិត្ត ខិតខំ ប្រឹង រៀន ប្រឹង...,negative


In [40]:
# Number of classes
num_classes = data['label'].nunique()
print("Number of classes:", num_classes)

# Optional: see which classes
print("Classes:", data['label'].unique())


Number of classes: 3
Classes: ['neutral' 'positive' 'negative']


In [41]:
# Count of samples per class
class_counts = data['label'].value_counts()
print(class_counts)


label
positive    5829
negative    3255
neutral      920
Name: count, dtype: int64


In [42]:
data.to_csv("../data/cleaned_data.csv", index=False)
